In [19]:
import numpy as np
import json
import warnings
import operator

import h5py
from keras.models import model_from_json
from keras import backend as K
from keras.utils import get_custom_objects

warnings.filterwarnings("ignore")

size_title = 18
size_label = 14
n_pred = 2


def read_file(file_path):
    with open(file_path, 'r') as data_file:
        data = json.loads(data_file.read())
    return data

def create_model(model_path):
    
    reverse_dictionary = dict((str(v), k) for k, v in dictionary.items())
    model_weights = list()
    weight_ctr = 0
    while True:
        try:
            d_key = "weight_" + str(weight_ctr)
            weights = trained_model.get(d_key).value
            model_weights.append(weights)
            weight_ctr += 1
        except Exception as exception:
            break
    # set the model weights
    loaded_model.set_weights(model_weights)
    return loaded_model, dictionary, reverse_dictionary, compatibile_tools


def verify_model(model, tool_sequence, labels, dictionary, reverse_dictionary, compatible_tools, class_weights, topk=20, max_seq_len=25):
    tl_seq = tool_sequence.split(",")
    last_tool_name = reverse_dictionary[str(tl_seq[-1])]
    last_compatible_tools = compatible_tools[last_tool_name]
    sample = np.zeros(max_seq_len)
    for idx, tool_id in enumerate(tl_seq):
        sample[idx] = int(tool_id)
    sample_reshaped = np.reshape(sample, (1, max_seq_len))

    tool_sequence_names = [reverse_dictionary[str(tool_pos)] for tool_pos in tool_sequence.split(",")]
    
    # predict next tools for a test path
    prediction = model.predict(sample_reshaped, verbose=0)
    
    weight_val = list(class_weights.values())
    weight_val = np.reshape(weight_val, (len(weight_val),))
    
    prediction = np.reshape(prediction, (prediction.shape[1],))
    
    prediction = prediction / float(np.max(prediction))
    
    prediction_pos = np.argsort(prediction, axis=-1)

    # get topk prediction
    topk_prediction_pos = prediction_pos[-topk:]
    topk_prediction_val = [int(prediction[pos] * 100) for pos in topk_prediction_pos]
    
    # read tool names using reverse dictionary
    pred_tool_ids = [reverse_dictionary[str(tool_pos)] for tool_pos in topk_prediction_pos if tool_pos > 0]
    actual_next_tool_ids = list(set(pred_tool_ids).intersection(set(last_compatible_tools.split(","))))

    pred_tool_ids_sorted = dict()
    for (tool_pos, tool_pred_val) in zip(topk_prediction_pos, topk_prediction_val):
        try:
            tool_name = reverse_dictionary[str(tool_pos)]
            if tool_name not in last_tool_name and tool_name in actual_next_tool_ids: #tool_name in actual_next_tool_ids and 
                pred_tool_ids_sorted[tool_name] = tool_pred_val
        except:
            continue
    pred_tool_ids_sorted = dict(sorted(pred_tool_ids_sorted.items(), key=lambda kv: kv[1], reverse=True))
    
    cls_wt = dict()
    usg_wt = dict()
    inv_wt = dict()
    ids_tools = dict()
    keys = list(pred_tool_ids_sorted.keys())
    for k in keys:
        try:
            cls_wt[k] = np.round(class_weights[str(data_dict[k])], 2)
            usg_wt[k] = np.round(usage_weights[k], 2)
            inv_wt[k] = np.round(inverted_weights[str(data_dict[k])], 2)
        except:
            continue
    print("Predicted tools: \n")
    print(pred_tool_ids_sorted)
    print()
    print("Class weights: \n")
    cls_wt = dict(sorted(cls_wt.items(), key=lambda kv: kv[1], reverse=True))
    print(cls_wt)
    print()
    print("Usage weights: \n")
    usg_wt = dict(sorted(usg_wt.items(), key=lambda kv: kv[1], reverse=True))
    print(usg_wt)
    print()
    total_usage_wt = np.mean(list(usg_wt.values()))
    print("Mean usage wt: %0.4f" % (total_usage_wt))
    print()
    print("Inverted weights: \n")
    inv_wt = dict(sorted(inv_wt.items(), key=lambda kv: kv[1], reverse=True))
    print(inv_wt)
    for key in pred_tool_ids_sorted:
        ids_tools[key] = dictionary[key]
    print()
    print("Tool ids")
    print(ids_tools)
    print("======================================")
    return cls_wt, usg_wt, inv_wt, pred_tool_ids_sorted

In [20]:
#base_path = "data/rnn_custom_loss/complete_training/"
base_path = "data/models/"

#model_path = base_path + "trained_model_19_09_1.h5"
model_path = base_path + "tool_recommendation_model.hdf5"

trained_model = h5py.File(model_path, 'r')
model_config = json.loads(trained_model.get('model_config').value)
class_weights = json.loads(trained_model.get('class_weights').value)
    
loaded_model = model_from_json(model_config)
dictionary = json.loads(trained_model.get('data_dictionary').value)
compatibile_tools = json.loads(trained_model.get('compatible_tools').value)
best_params = json.loads(trained_model.get('best_parameters').value)

model, dictionary, reverse_dictionary, compatibile_tools = create_model(model_path)

print(reverse_dictionary)

{'1': 'fastq_to_fasta_python', '2': 'tophat2', '3': 'eukaryotic_ncbi_submission', '4': 'gemini_windower', '5': 'antismash', '6': 'ctb_remSmall', '7': 'glimmer_build-icm', '8': 'bam_to_sam', '9': 'cshl_sort_header', '10': 'cufflinks', '11': 'gatk2_print_reads', '12': 'sam_to_bam', '13': 'blockclust', '14': 'Grouping1', '15': 'r_correlation_matrix', '16': 'ncbi_blastn_wrapper', '17': 'CONVERTER_interval_to_bedstrict_0', '18': 'ctb_compound_convert', '19': 'sort1', '20': 'vcftools_isec', '21': 'ctb_online_data_fetch', '22': 'cor2', '23': 'fasta_filter_by_length', '24': 'cshl_grep_tool', '25': 'deeptools_bamFingerprint', '26': 'cuffdiff', '27': 'fastq_quality_trimmer', '28': 'sam_bw_filter', '29': 'gatk2_realigner_target_creator', '30': 'CONVERTER_bed_to_gff_0', '31': 'random_lines1', '32': 'bams2ratio', '33': 'meme_meme', '34': 'in_tool', '35': 'rgPicardMarkDups', '36': 'deseq2', '37': 'bismark_bowtie2', '38': 'gemini_interactions', '39': 'DatamashTranspose', '40': 'gops_join_1', '41': 's

In [24]:
topk = 30
tool_seq = "244"
class_wt, usage_wt, inverse_wt, pred_tools = verify_model(model, tool_seq, "", dictionary, reverse_dictionary, compatibile_tools, class_weights, topk)

Predicted tools: 

{'tab2fasta': 100, 'cor2': 100, 'Show tail1': 100, 'cshl_word_list_grep': 100, 'CONVERTER_interval_to_bgzip_0': 100, 'CONVERTER_bedgraph_to_bigwig': 100, 'Grep1': 100, 'sort1': 100, 'wig_to_bigWig': 100, 'CONVERTER_interval_to_bed_0': 100, 'Add_a_column1': 99, 'join1': 99, 'wc_gnu': 99, 'Filter1': 99, 'XY_Plot_1': 99, 'cshl_uniq_tool': 99, 'cshl_sort_header': 99, 'Paste1': 99, 'addValue': 99, 'Convert characters1': 99, 'Summary_Statistics1': 99, 'gops_intersect_1': 99, 'Count1': 99, 'bedtools_intersectbed_bam': 99, 'methtools_filter': 99, 'cat1': 99, 'r_correlation_matrix': 99, 'cshl_find_and_replace': 99, 'bedtools_genomecoveragebed_bedgraph': 99, 'Remove beginning1': 99}

Class weights: 

{}

Usage weights: 

{}

Mean usage wt: nan

Inverted weights: 

{}

Tool ids
{'tab2fasta': 72, 'cor2': 22, 'Show tail1': 238, 'cshl_word_list_grep': 96, 'CONVERTER_interval_to_bgzip_0': 145, 'CONVERTER_bedgraph_to_bigwig': 186, 'Grep1': 185, 'sort1': 19, 'wig_to_bigWig': 183, 'CO

In [17]:
class_weights['294']

4.556972389179466